In [5]:
import pygame
import time
import random
pygame.font.init()

WIDTH, HEIGHT = 600, 500
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Space Shooter Game")


# Load images
RED_SPACE_SHIP = pygame.transform.scale(pygame.image.load("enemy3.png"),(60,28))
GREEN_SPACE_SHIP = pygame.transform.scale(pygame.image.load("enemy1.png"),(64,44))
BLUE_SPACE_SHIP = pygame.transform.scale(pygame.image.load("enemy2.png"),(61,33))

# Player player
SPACE_SHIP = pygame.transform.scale(pygame.image.load("spaceship1.png"),(50,50))

# Lasers
RED_LASER = pygame.transform.scale(pygame.image.load("pixel_laser_red.png"),(65,75))
GREEN_LASER = pygame.transform.scale(pygame.image.load("pixel_laser_green.png"),(65,75))
BLUE_LASER = pygame.transform.scale(pygame.image.load("pixel_laser_blue.png"),(65,75))
YELLOW_LASER = pygame.transform.scale(pygame.image.load("pixel_laser_yellow.png"),(65,75))

# Background
BG = pygame.transform.scale(pygame.image.load("background-black.png"), (WIDTH, HEIGHT))

score = 0
#multiplayer
multiplayer = False
player10 = True
player20 = True

class Laser:
    def __init__(self, x, y, img):
        self.x = x
        self.y = y
        self.img = img
        self.mask = pygame.mask.from_surface(self.img)

    def draw(self, window):
        window.blit(self.img, (self.x, self.y))

    def move(self, vel):
        self.y += vel

    def off_screen(self, height):
        return not(self.y <= height and self.y >= 0)

    def collision(self, obj):
        return collide(self, obj)
    
class Ship:
    COOLDOWN = 30

    def __init__(self, x, y, health=100):
        self.x = x
        self.y = y
        self.health = health
        self.ship_img = None
        self.laser_img = None
        self.lasers = []
        self.cool_down_counter = 0

    def draw(self, window):
        window.blit(self.ship_img, (self.x, self.y))
        for laser in self.lasers:
            laser.draw(window)

    def move_lasers(self, vel, obj):
        self.cooldown()
        for laser in self.lasers:
            laser.move(vel)
            if laser.off_screen(HEIGHT):
                self.lasers.remove(laser)
            elif laser.collision(obj):
                obj.health -= 10
                self.lasers.remove(laser)

    def cooldown(self):
        if self.cool_down_counter >= self.COOLDOWN:
            self.cool_down_counter = 0
        elif self.cool_down_counter > 0:
            self.cool_down_counter += 1

    def shoot(self):
        if self.cool_down_counter == 0:
            laser = Laser(self.x-8, self.y-30, self.laser_img)
            self.lasers.append(laser)
            self.cool_down_counter = 1

    def get_width(self):
        return self.ship_img.get_width()

    def get_height(self):
        return self.ship_img.get_height()
    

class Player(Ship):
    
    def __init__(self, x, y, health=100):
        super().__init__(x, y, health) 
        self.ship_img = SPACE_SHIP
        self.laser_img = YELLOW_LASER
        self.mask = pygame.mask.from_surface(self.ship_img)
        self.max_health = health

    def move_lasers(self, vel, objs):
        global score
        self.cooldown()
        for laser in self.lasers:
            laser.move(vel)
            if laser.off_screen(HEIGHT):
                self.lasers.remove(laser)
            else:
                for obj in objs:
                    if laser.collision(obj):
                        objs.remove(obj)
                        score += 1
                        if laser in self.lasers:
                            self.lasers.remove(laser)

    def draw(self, window):
        super().draw(window)
        self.healthbar(window)

    def healthbar(self, window):
        pygame.draw.rect(window, (255,0,0), (self.x, self.y + self.ship_img.get_height() + 10, self.ship_img.get_width(), 10))
        pygame.draw.rect(window, (0,255,0), (self.x, self.y + self.ship_img.get_height() + 10, self.ship_img.get_width() * (self.health/self.max_health), 10))


class Enemy(Ship):
    COLOR_MAP = {
                "red": (RED_SPACE_SHIP, RED_LASER),
                "green": (GREEN_SPACE_SHIP, GREEN_LASER),
                "blue": (BLUE_SPACE_SHIP, BLUE_LASER)
                }

    def __init__(self, x, y, color, health=100):
        super().__init__(x, y, health)
        self.ship_img, self.laser_img = self.COLOR_MAP[color]
        self.mask = pygame.mask.from_surface(self.ship_img)

    def move(self, vel):
        self.y += vel

    def shoot(self):
        if self.cool_down_counter == 0:
            laser = Laser(self.x-12, self.y, self.laser_img)
            self.lasers.append(laser)
            self.cool_down_counter = 1


def collide(obj1, obj2):
    offset_x = obj2.x - obj1.x
    offset_y = obj2.y - obj1.y
    return obj1.mask.overlap(obj2.mask, (offset_x, offset_y)) != None

def main():
    run = True                                     #constant attributes
    FPS = 60
    level = 0
    lives = 5
    enemies = []
    wave_length = 5
    enemy_vel = 1
    player_vel = 5
    laser_vel = 6
    global score
    score = 0
    
    if multiplayer:                                #players intialization
        player2 = Player(120, 375)
        player1 = Player(415, 375)  
    else:
        player = Player(275, 375)
           
    main_font = pygame.font.SysFont("comicsans", 25)
    lost_font = pygame.font.SysFont("comicsans", 40)
    smallText = pygame.font.SysFont("comicsansms",20)
         
    clock = pygame.time.Clock()

    lost = False
    lost_count = 0

    def redraw_window():
        WIN.blit(BG, (0,0))
        
        if multiplayer:                                                               #Display lives left & players
            lives_label = main_font.render(f"Team Lives: {lives}", 1, (255,255,255))
            if player10:
                player1.draw(WIN)        #player1
            if player20:
                player2.draw(WIN)        #player2
        else:
            lives_label = main_font.render(f"Lives: {lives}", 1, (255,255,255))
            player.draw(WIN)
        
        WIN.blit(lives_label, (10, 10))
        level_label = main_font.render(f"Level: {level}", 1, (255,255,255))
        total = main_font.render(f"Score: {score}", 1, (255,255,255))
        WIN.blit(level_label, (WIDTH - level_label.get_width() - 15, 10))
        WIN.blit(total, (WIDTH - level_label.get_width() - 15, 35))

        for enemy in enemies:
            enemy.draw(WIN)

        if lost:                                                                 #Lost messages..
            lost_label = lost_font.render("You Lost!!", 1, (255,255,255))
            total = smallText.render(f"Score:{score}", 1, (255,255,255))
            WIN.blit(lost_label, (WIDTH/2 - lost_label.get_width()/2, 250))
            WIN.blit(total, (WIDTH/2 - total.get_width()/2, 380))

        pygame.display.update()
    
    global player10, player20
    
    while run:                                                                #Playing program
        clock.tick(FPS)
        redraw_window()

        if multiplayer:                                                             #for multiplayer->
            
            if lives <= 0 or (player1.health <= 0 and player2.health <= 0):
                lost = True
                lost_count += 1
                
            if player1.health <= 0:
                player10 = False
            if player2.health <= 0:
                player20 = False
                   
            keys = pygame.key.get_pressed()
            if player10:
                if keys[pygame.K_LEFT] and player1.x - player_vel > 0: # left
                    player1.x -= player_vel
                if keys[pygame.K_RIGHT] and player1.x + player_vel + player1.get_width() < WIDTH: # right
                    player1.x += player_vel
                if keys[pygame.K_UP] and player1.y - player_vel > 0: # up
                    player1.y -= player_vel
                if keys[pygame.K_DOWN] and player1.y + player_vel + player1.get_height() + 15 < HEIGHT: # down
                    player1.y += player_vel
                if keys[pygame.K_SPACE]:
                    player1.shoot()
                    
            if keys[pygame.K_ESCAPE]:
                paused()
                
            if player20:
                if keys[pygame.K_a] and player2.x - player_vel > 0: # left
                    player2.x -= player_vel
                if keys[pygame.K_d] and player2.x + player_vel + player2.get_width() < WIDTH: # right
                    player2.x += player_vel
                if keys[pygame.K_w] and player2.y - player_vel > 0: # up
                    player2.y -= player_vel
                if keys[pygame.K_s] and player2.y + player_vel + player2.get_height() + 15 < HEIGHT: # down
                    player2.y += player_vel
                if keys[pygame.K_LCTRL]:
                    player2.shoot()
                    
            if lost:
                if lost_count > FPS * 5:
                    run = False
                else:
                    continue
        
            for enemy in enemies[:]:
                enemy.move(enemy_vel)
                enemy.move_lasers(laser_vel, player1)
                enemy.move_lasers(laser_vel, player2)

                if random.randrange(0, 2*60) == 1:
                    enemy.shoot()
                
                if player10:
                    if collide(enemy, player1):
                        player1.health -= 10
                        enemies.remove(enemy)
                if player20:
                    if collide(enemy, player2):
                        player2.health -= 10
                        enemies.remove(enemy)
                        
                if enemy.y + enemy.get_height() > HEIGHT:
                    lives -= 1
                    enemies.remove(enemy)

            player1.move_lasers(-laser_vel, enemies)
            player2.move_lasers(-laser_vel, enemies)
                    
        else:                                                               #Single player->
            
            if lives <= 0 or player.health <= 0:
                lost = True
                lost_count += 1
                
            keys = pygame.key.get_pressed()
            if keys[pygame.K_a] and player.x - player_vel > 0: # left
                player.x -= player_vel
            if keys[pygame.K_d] and player.x + player_vel + player.get_width() < WIDTH: # right
                player.x += player_vel
            if keys[pygame.K_w] and player.y - player_vel > 0: # up
                player.y -= player_vel
            if keys[pygame.K_s] and player.y + player_vel + player.get_height() + 15 < HEIGHT: # down
                player.y += player_vel
            if keys[pygame.K_SPACE]:
                player.shoot()
            if keys[pygame.K_ESCAPE]:
                paused()   
            if keys[pygame.K_p]:
                paused()
                
            
            if lost:
                if lost_count > FPS * 5:
                    run = False
                else:
                    continue
        
            for enemy in enemies[:]:
                enemy.move(enemy_vel)
                enemy.move_lasers(laser_vel, player)

                if random.randrange(0, 2*60) == 1:
                    enemy.shoot()

                if collide(enemy, player):
                    player.health -= 10
                    enemies.remove(enemy)
                elif enemy.y + enemy.get_height() > HEIGHT:
                    lives -= 1
                    enemies.remove(enemy)
                
            player.move_lasers(-laser_vel, enemies)
            

        if len(enemies) == 0:                                                   #Common run for both
            level += 1
            wave_length += 5
            for i in range(wave_length):
                enemy = Enemy(random.randrange(50, WIDTH-80), random.randrange(-1500, -100), random.choice(["red", "blue", "green"]))
                enemies.append(enemy)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                

def main_menu():                                                               #Main menu shown before game start
    title_font = pygame.font.SysFont("comicsans", 25)
    
    run = True
    while run:
        WIN.blit(BG, (0,0))
        title_label = title_font.render("Press the mouse to begin...", 1, (255,255,255))
        WIN.blit(title_label, (WIDTH/2 - title_label.get_width()/2, 250))
        back_key = title_font.render("backspace --> main menu", 1, (255,255,255))
        WIN.blit(back_key, (WIDTH - back_key.get_width() - 10, 10))
        global player10,player20
        
        if multiplayer:
            player10 = True
            player20 = True
            player1 = Player(130, 375)
            player2 = Player(415, 375)
            player1.draw(WIN)       
            player2.draw(WIN)        
        else:
            player = Player(275, 375)
            player.draw(WIN)
            
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            keys = pygame.key.get_pressed()
            if keys[pygame.K_ESCAPE]:
                run = False 
            if keys[pygame.K_BACKSPACE]:
                run = False
            if event.type == pygame.MOUSEBUTTONDOWN:
                main()
            
            
def option():                                                                  #key binding show func 
    title_font = pygame.font.SysFont("comicsans", 25)
    back_key = title_font.render("backspace --> main menu", 1, (255,255,255))
    run = True
    while run:
        AR = pygame.transform.scale(pygame.image.load("option_menu3.png"),(600,500)) 
        WIN.blit(AR, (0,0))
        WIN.blit(back_key, (WIDTH - back_key.get_width() - 10, 10))
        pygame.display.update()
        for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                keys = pygame.key.get_pressed()
                if keys[pygame.K_BACKSPACE]:
                    run = False
                if keys[pygame.K_ESCAPE]:
                    run = False 
    
                    
def paused():                                                             #paused func
    largeText = pygame.font.SysFont("comicsansms",75)
    smallText = pygame.font.SysFont("comicsansms",20)
    text = largeText.render("Paused",1,(255,255,255))
    text2= smallText.render("Press 'o' to continue",1,(255,255,255))
    back_key = smallText.render("backspace --> main menu", 1, (255,255,255))
    WIN.blit(back_key, (WIDTH - back_key.get_width() - 10, 55))
    WIN.blit(text, (WIDTH/2 - text.get_width()/2, 190))
    WIN.blit(text2, (WIDTH/2 - text2.get_width()/2, 290))
    pygame.display.update()
    pause = True
    
    while pause:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                    
        keys = pygame.key.get_pressed() 
        if keys[pygame.K_o]:
            pause = False
        
        if keys[pygame.K_BACKSPACE]:
            intro()
    
       
def button(msg,x,y,w,h,ic,ac,action=None):                             #button display & action func
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    global multiplayer
    
    smallText = pygame.font.SysFont("comicsansms",20)
    text = smallText.render(msg,1,(255,255,255)) 
     
    if x+w > mouse[0] > x and y+h > mouse[1] > y:
        pygame.draw.rect(WIN, ac,(x,y,w,h))

        if click[0] == 1 and action == "start":
            #multiplayer = False
            main_menu()
            WIN.blit(BG, (0,0))
        if click[0] == 1 and action == "multiplayer":
            
            multiplayer = True
            main_menu()
            multiplayer = False
            WIN.blit(BG, (0,0))
        if click[0] == 1 and action == "options":
            option()
            intro()
        if click[0] == 1 and action == "exit":
            pygame.quit()
        
    else:
        pygame.draw.rect(WIN, ic,(x,y,w,h))

    WIN.blit(text, (WIDTH/2 - text.get_width()/2, ((y+(h/2))-17)) )
    pygame.display.update()               
                
                
def intro():                                                         #Intro display with buttons
    
    WIN.blit(BG, (0,0))
    
    main_font = pygame.font.SysFont("comicsans", 28)
    text = main_font.render("Ashankhere ", 1, (255,255,255))
    WIN.blit(text, (90, 10))
     
    pygame.display.update()
    run = True
    while run:      
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False    
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_w:
                    main_menu()
                    run = False
                if event.key == pygame.K_a:
                    run = False

        button("Single player",215,100,170,50,(0,0,0),(90,90,90),"start")
        button("Multiplayer",200,200,200,50,(0,0,0),(90,90,90),"multiplayer")
        button("Options",240,300,120,50,(0,0,0),(90,90,90),"options")
        button("Exit",250,400,100,50,(0,0,0),(90,90,90),"exit")
                    
                    
    pygame.quit()

intro()                                                                     #run